## Imports + Setup

In [ ]:
import gzip
import json
from typing import List
from collections import Counter, defaultdict
from tqdm import tqdm
import os
import inspect_ai
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv
import torch as t
import subprocess
import contextlib
import shutil
import ast
import textwrap
import copy
from openai import OpenAI

In [ ]:
from inspect_ai import Task, task
from inspect_ai.dataset import Sample, hf_dataset
from inspect_ai.util import ExecResult, sandbox
from inspect_ai.scorer import CORRECT, INCORRECT, Score, Scorer, Target, accuracy, scorer, stderr
from inspect_ai.solver import TaskState, generate
from inspect_ai.model import get_model
from inspect_ai.log import read_eval_log
import re

In [ ]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])
client = OpenAI()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
IMPORT_HELPER = {
    "python": [
        "import math",
        "import re",
        "import sys",
        "import copy",
        "import datetime",
        "import itertools",
        "import collections",
        "import heapq",
        "import statistics",
        "import functools",
        "import hashlib",
        "import numpy",
        "import numpy as np",
        "import string",
        "from typing import *",
        "from collections import *",
    ],
    "go"    : [
        "math",
        "strings",
        "fmt",
        "strconv",
        "time",
        "bytes",
        "regexp",
        "sort",
        "math/rand",
        "crypto/md5",
    ],
    "cpp"   : [
        "#include<stdlib.h>",
        "#include<algorithm>",
        "#include<math.h>",
        "#include<stdio.h>",
        "#include<vector>",
        "#include<string>",
        "#include<climits>",
        "#include<cstring>",
        "#include<iostream>",
        "#include <numeric>",
        "#include <sstream>",
        "#include <stack>",
    ],
}

# instruction prepended to code problem
HUMANEVAL_INSTRUCTION = """
Read the following function signature and docstring, and fully implement
the function described. Your response should only contain the code for
this function.\n
"""

CODE_EXTRACTION_INSTRUCTION = """
Here is a section of code. Follow these instructions to extract the function body.
First, if present, remove any initial package names, import statements, and comments. 
Next, remove the function signature of the first function you see. 
If there is a `main` function present, delete that entire function. 
However, if there are any other functions present, leave them as they are.

Your response should only contain the remaining block of code. \n
"""

LANG_PREFIX = {
    "cpp"          : "// language: C++",
    "java"         : "// language: Java",
    "js"           : "// language: JavaScript",
    "javascript"   : "// language: JavaScript",
    "go"           : "// language: Go",
    "python"       : "# language: Python",
}

In [ ]:
# model = get_model(
#         'hf/meta-llama/Llama-3.1-8B-Instruct', 
#         device = 'auto',
#         torch_dtype=t.bfloat16,
# )

### Language Setup

In [ ]:
# JavaScript
#     `apt-get update`  
#     `apt-get install -y curl`  
#     `curl -fsSL https://deb.nodesource.com/setup_lts.x | bash -`  
#     `apt-get install -y nodejs`  
#     verify installation: `node -v` // `npm -v` // `node -e "console.log('Node.js is working')"`  

In [ ]:
# Java
#     `apt-get update`  
#     `apt-get install -y openjdk-21-jdk`  
# Verify with:  
#     `java -version`  
#     `javac -version` 

In [ ]:
# Go
#     `cd ~/`  
#     `GO_VERSION=1.24.5`  
#     `wget https://go.dev/dl/go${GO_VERSION}.linux-amd64.tar.gz`  
#     `rm -rf /usr/local/go`  
#     `tar -C /usr/local -xzf go${GO_VERSION}.linux-amd64.tar.gz`  
#     `echo 'export PATH=$PATH:/usr/local/go/bin' >> ~/.bashrc`  
#     `export PATH=$PATH:/usr/local/go/bin`  
#     `go version`  

In [ ]:
# C++
#     apt-get update
#     apt-get install -y g++
#     apt-get install -y build-essential
#     apt-get install libboost-all-dev
#     apt-get install libssl-dev


## Data Processing

In [ ]:
def stream_jsonl_all(filename: str):
    results = []
    fp = gzip.open(open(filename, "rb"), "rt")
    for line in fp:
        if any(not x.isspace() for x in line):
            results.append(json.loads(line))
    fp.close()

    return results

In [ ]:
python_content = stream_jsonl_all('data/python_data.gz')
cpp_content = stream_jsonl_all('data/cpp_data.gz')
go_content = stream_jsonl_all('data/go_data.gz')
java_content = stream_jsonl_all('data/java_data.gz')
js_content = stream_jsonl_all('data/js_data.gz')
content = [python_content, cpp_content, go_content, java_content, js_content]

In [ ]:
generations = stream_jsonl_all('data/python_generations.gz')
generations[0]['generation']

In [ ]:
for lang in content:
    print(lang[0].keys())

## Eval Pipeline

### Extract Code

In [1]:
def identify_codeblock(completion: str) -> str:
    pattern_1 = re.compile(r"```(?:python|javascript|java|cpp|go)\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)

    if matches == []:
        return completion
    else:
        return matches[0]

In [ ]:
def remove_signature(completion):
    prompt = CODE_EXTRACTION_INSTRUCTION + completion
    response = client.responses.create(
        model='gpt-4.1-mini',
        input=prompt
    )

    text_out = response.output[-1].content[0].text
    return text_out

#### CPP

In [ ]:
def balance_brackets_cpp(completion):
    if '{' in completion[:3]:
        completion = completion.replace('{', "", 1)

    difference = completion.count('{') - completion.count('}') 
    if difference == 0: 
        completion += '}'
        difference -= 1
    if difference != -1: 
        print('brackets ain\'t balancing')
    
    return completion

def find_code_cpp(completion: str) -> str:
    processed = remove_signature(completion)
    processed = identify_codeblock(processed)
    processed = balance_brackets_cpp(processed)

    return processed

In [ ]:
def get_final_cpp(state, completion):
    imports = ''
    for s in IMPORT_HELPER['cpp']:
        if s not in state.metadata['prompt']:
            imports += s + '\n'

    code = imports + "\n" + state.metadata['prompt'] + completion + "\n" + state.metadata['test']
    
    return code

#### Go

In [ ]:
def balance_brackets_go(completion):
    if '{' in completion[:3]:
        completion = completion.replace('{', "", 1)

    difference = completion.count('{') - completion.count('}') 
    if difference == 0: 
        completion += '\n}'
        difference -= 1
    if difference != -1: 
        print('brackets ain\'t balancing')
    
    return completion

def find_code_go(completion: str) -> str:
    processed = remove_signature(completion)
    processed = identify_codeblock(processed)
    processed = balance_brackets_go(processed)

    return processed

In [ ]:
def get_final_go(state, completion):
    import_string = state.metadata['import']
    prompt = state.metadata['prompt'].replace(import_string, '')

    test = state.metadata['test']
    test_setup = state.metadata['test_setup']
    other_pkgs = []

    for pkg in IMPORT_HELPER['go']:
        if pkg not in test_setup:
            p = pkg.split('/')[-1]
            if p + '.' in completion:    
                other_pkgs.append(f"\"{pkg}\"")
    if other_pkgs:
        import_other_pkgs = "import (\n" + "    ".join([p + "\n" for p in other_pkgs]) + ")"
        final_code = test_setup + "\n" + import_other_pkgs + "\n" + prompt + completion + "\n" + test
    else:
        final_code = test_setup + "\n" + prompt + completion + "\n" + test

    return final_code

#### Java

In [ ]:
def remove_unindented(lines, full_func):
    while (not lines[0]) or (lines[0] == lines[0].lstrip()):
        full_func = True
    return lines, full_func

def remove_signature_java(lines, full_func):
    '''
    deals with three cases
    a) first lines are comments, before method header
    b) first line is method-header
    c) b), but with incorrect indentation? so header had accidentally been removed?
    '''
    if not (full_func or lines[0].lstrip()[:6] == 'public'):
        return lines
    
    removed = False
    og_lines = copy.deepcopy(lines)
    
    while not removed:
        if lines == []:
            return og_lines
        
        line = lines.pop(0)
        if line.lstrip()[:6] == 'public':
            removed = True
        
    return lines

def balance_brackets_java(processed):
    difference = processed.count('{') - processed.count('}') + 2
    assert difference in [0, 1, 2], 'brackets ain\'t balancing'
    return processed + ('}' * difference)

def find_code_java(completion: str) -> str:
    code = identify_codeblock(completion)
    lines = code.splitlines()
    lines, full_func = remove_unindented(lines, False)
    lines = remove_signature_java(lines, full_func)

    processed = '\n'.join(lines)
    processed = balance_brackets_java(processed)
    
    return processed

In [ ]:
def get_final_java(state, completion):
    final_code = state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
    return final_code

#### Java [archived]

In [ ]:
# def remove_method_header(lines,):
#     '''
#     deals with three cases
#     a) first lines are comments, before method header
#     b) first line is method-header
#     c) b), but with incorrect indentation? so header had accidentally been removed?
#     '''
#     removed = False
#     og_lines = copy.deepcopy(lines)
    
#     while not removed:
#         if lines == []:
#             return og_lines
        
#         line = lines.pop(0)
#         if line.lstrip()[:6] == 'public':
#             removed = True
        
#     return lines

# def balance_brackets(lines):
#     final_code = '\n'.join(lines)
#     difference = final_code.count('{') - final_code.count('}') + 2
#     assert difference in [0, 1, 2], 'brackets ain\'t balancing'
#     return final_code + ('}' * difference)

# def remove_unindented(lines, full_func):
#     import_statements = []
#     while (not lines[0]) or (lines[0] == lines[0].lstrip()):
#         line = lines.pop(0)
#         full_func = True
#         if 'import' in line: import_statements.append(line)
    
#     return lines, import_statements, full_func

# def find_code_java(completion: str) -> str:
#     code = identify_codeblock(completion)
#     lines = code.splitlines()

#     lines, import_statements, full_func = remove_unindented(lines, False)

#     if full_func or lines[0].lstrip()[:6] == 'public':
#         lines = remove_method_header(lines)

#     processed_completion = balance_brackets(lines)
#     import_statements = '\n'.join(import_statements)
    
#     return processed_completion, import_statements

In [ ]:
# def get_final_java(imports, state, completion):
#     final_code = imports + '\n' + state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
#     return final_code

#### JavaScript

In [ ]:
def remove_signature_js(code: str) -> str:
    lines = code.splitlines()
    found = False
    for line in lines:
        if line.lstrip().startswith('const'):
            lines.remove(line)
            found = True
            break  

    if not found: 
        print('error extracting function body')
        return 'errormsg'
    return "\n".join(lines)

def find_code_js(completion: str) -> str:
    processed = identify_codeblock(completion)
    processed = remove_signature_js(processed)
    return processed

In [ ]:
def get_final_js(state, completion):
    final_code = state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
    return final_code

#### Python

In [ ]:
def remove_signature_python(code: str) -> str:
    try:
        tree = ast.parse(code)
        for node in tree.body:
            if not isinstance(node, ast.FunctionDef):
                continue
            code_lines = code.splitlines()
            start = node.body[0].lineno - 1
            end = node.body[-1].end_lineno
            body_lines = code_lines[start:end]
            return "\n".join(body_lines)

    except Exception as e:
        print(f"Error extracting function body: {e}")
        return "errormsg"
    
def find_code_python(completion):
    processed = identify_codeblock(completion)
    processed = remove_signature_python(processed)
    return processed

In [ ]:
def get_final_python(state, completion):
    imports = "\n".join(IMPORT_HELPER["python"]) + "\n"
    final_code = imports + state.metadata['prompt'] + completion + "\n" + state.metadata['test'] + "\n"
    return final_code

### Routing

In [ ]:
def find_code(completion, lang: str):
    lang = lang.lower()
    pass

def get_final(state, completion, lang: str):
    pass